In [7]:
%matplotlib inline
!pip install seaborn

You are using pip version 9.0.3, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
# Model architecture
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, Flatten, SpatialDropout1D
from keras.models import Sequential
from keras import optimizers
from keras.callbacks import Callback

# General packages
import numpy as np
import os
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Data preparation and validation packages
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# Jupyter interactive plotting
from IPython.display import clear_output

from components.dataToolkit import dataToolkit, LABEL_MAP

In [9]:
#call back functions
#real time plotting
class PlotLosses(Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.accuracies = []
        self.val_losses = []
        self.val_accuracies = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.accuracies.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_accuracies.append(logs.get('val_acc'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.accuracies, label="accuracy")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.plot(self.x, self.val_accuracies, label="val_accuracy")
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show();
        
plot_losses = PlotLosses()

In [10]:
dtk = dataToolkit()
data, labels = dtk.getAllTimeSeries()


In [6]:
rescaled_data = data
labels
data

array([[ 4.80499620e-20, -2.11897128e-19, -4.38822667e-19, ...,
         1.63988049e-19, -4.80369117e-19, -6.47482382e-20],
       [-1.03171798e-18, -1.05989688e-18, -1.05439710e-18, ...,
        -1.43545469e-18, -1.41802348e-18, -1.37340593e-18],
       [ 2.46173982e-20, -9.62740485e-21,  4.04646469e-21, ...,
        -3.49280950e-19, -3.03520695e-19, -3.02714835e-19],
       ...,
       [            nan,             nan,             nan, ...,
                    nan,             nan,             nan],
       [            nan,             nan,             nan, ...,
                    nan,             nan,             nan],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [13]:
# Let's shuffle data
ind = np.random.permutation(rescaled_data.shape[0])
rescaled_data = np.take(rescaled_data, ind, axis=0)
labels = np.take(labels, ind)

16384

In [14]:
# Let's convert labels into one hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
labels = labels.reshape(labels.shape[0], 1)
labels
targets = onehot_encoder.fit_transform(labels)
targets

array([3., 3., 3., ..., 3., 3., 0.])

In [ ]:
# Split the data for training and validation
x_train, x_val, y_train, y_val = train_test_split(rescaled_data, targets, test_size=0.3)